# (17.0) Описание решаемой задачи

На основании разработанного алгоритма и определенной дельты осуществить расчет цепочек для всех найденных айсбергов

# (17.1) Установка всех необходимых пакетов¶

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

# (17.2) Определение функций¶

In [3]:
# перевод времени в формат datetime
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# восстановление стакана заявок и расчет основных метрик
def GlassBuild(for_lob):
    
    #формируем очередь заявок
    
    # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mkt_orders = for_lob['PRICE']!=0
    # таблица выставленных заявок
    plc_orders = for_lob[['ORDERNO','BUYSELL','PRICE','VOLUME']][(for_lob ['ACTION']==1) & no_mkt_orders]
    # таблица отмененных заявок
    wdr_orders = for_lob[['ORDERNO','VOLUME']][(for_lob ['ACTION'] == 0) & no_mkt_orders]
    # таблица исполненных заявок
    trd_orders = for_lob [['ORDERNO','VOLUME']][(for_lob['ACTION']==2) & no_mkt_orders]
    trd_orders = trd_orders.groupby(['ORDERNO']).sum()
    trd_orders['ORDERNO'] = trd_orders.index
    trd_orders.index=range(trd_orders.shape[0])
    # собираем все типы заявок в один датафрейм - очередь заявок
    lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
    lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
    lob_almost = lob_almost.fillna(0)
    lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
    # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]
    
    # строим стакан
    
    lob = lob_almost.pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    
    # считаем метрики ликвидности
    
    # расчет лучшей цены на покупку
    bid_price = lob[lob['B'] > 0].index[0]
    # расчет объема на уровне лучшей цены на покупку
    bid_volume = lob['B'][bid_price]
    # расчет общего объема на стороне покупки
    bid_depth = lob['B'].sum()
    # расчет объема на уровне 10 лучших цен на стороне покупки
    b_10 = lob[lob['B'] > 0].index[0:10]
    volume_b_10 = 0
    for i in b_10:
        volume_b_10 = volume_b_10 + lob['B'][i]
    # расчет лучшей цены на продажу
    ask_price = lob[lob['S'] > 0].index[-1]
    # расчет объема на уровне лучшей цены на продажу
    ask_volume = lob['S'][ask_price]
    # расчет общего объема на стороне продажи
    ask_depth = lob['S'].sum()
    # расчет объема на уровне 10 лучших цен на стороне продажи
    s_10 = lob[lob['S'] > 0].index[-10:]
    volume_s_10 = 0
    for i in s_10:
        volume_s_10 = volume_s_10 + lob['S'][i]
    
    liquidity = [bid_price, bid_volume, bid_depth, volume_b_10, ask_price, ask_volume, ask_depth, volume_s_10]
     
    return(liquidity)

# проверка, что время выставления каждой следующей заявки в цепочке больше времени исполнения предыдущей, если нет, то такая пара исключается из цепочки
# используется при формировании цепочки перед айсбергом
def ChainPairPrev(list_of_orderno):
    
    list_of_pairs = []
    
    for i in range(len(list_of_orderno)-1):
    
        cur_orderno = list_of_orderno[i]
        prev_orderno = list_of_orderno[i+1]
        cur_time_place_1 = orders_instr.index[(orders_instr.ORDERNO == cur_orderno)&(orders_instr.ACTION == 1)].to_list()[0]
        cur_time_place = orders_instr.loc[cur_time_place_1,'TIME']
        prev_time_trd_1 = orders_instr.index[(orders_instr.ORDERNO == prev_orderno)&(orders_instr.ACTION == 2)].to_list()[-1]
        prev_time_trd = orders_instr.loc[prev_time_trd_1,'TIME']
    
        if (cur_time_place >= prev_time_trd) and ((cur_time_place - prev_time_trd) <= delta):

            list_of_pairs.append([cur_orderno, prev_orderno])
            
        else:
            
            break
    
    list_of_orderno = [orderno]
    
    if len(list_of_pairs) != 0:

        for i in range(len(list_of_pairs)):
    
            list_of_orderno.append(list_of_pairs[i][1])
       
    return(list_of_orderno)

# проверка, что время выставления каждой следующей заявки в цепочке больше времени исполнения предыдущей, если нет, то такая пара исключается из цепочки
# используется при формировании цепочки после айсберга
def ChainPairSuc(list_of_orderno):
    
    list_of_pairs = []
    
    for i in range(len(list_of_orderno)-1):
    
        cur_orderno = list_of_orderno[i]
        suc_orderno = list_of_orderno[i+1]
        cur_time_trd_1 = orders_instr.index[(orders_instr.ORDERNO == cur_orderno)&(orders_instr.ACTION == 2)].to_list()[-1]
        cur_time_trd = orders_instr.loc[cur_time_trd_1,'TIME']
        suc_time_place_1 = orders_instr.index[(orders_instr.ORDERNO == suc_orderno)&(orders_instr.ACTION == 1)].to_list()[0]
        suc_time_place = orders_instr.loc[suc_time_place_1,'TIME']
    
        if (suc_time_place >= cur_time_trd) and ((suc_time_place - cur_time_trd) <= delta):

            list_of_pairs.append([cur_orderno, suc_orderno])
            
        else:
            
            break
    
    list_of_orderno = [orderno]
    
    if len(list_of_pairs) != 0:

        for i in range(len(list_of_pairs)):
    
            list_of_orderno.append(list_of_pairs[i][1])
       
    return(list_of_orderno)

# (17.3) Список всех используемых датафреймов

In [4]:
# orders - ордерлог на заданную дату
# iceberg_all - список всех найденных айсбергов
# orders_instr - ордерлог по инструменту
# ice_instr - список айсбергов по одному инструменту за один день

# (17.4) Определение входных параметров

In [7]:
# создаем список из всех названий файлов с данными (из них в дальнейшем можно извлечь дату)
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [347]:
# создаем список из инструментов
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [292]:
d = orderlogs[15] 
date = int(d[8:16]) #20190304
ticker = tickers[8] #YNDX
delta = dt.timedelta(microseconds=10000)

# (17.5) Чтение данных

In [293]:
%%time
# преобразуем time в формат datetime
# orders - ордерлог за день
# instr_orders - ордерлог на заданную дату по выбранному инструменту на момет времени t
orders = pd.read_csv(d, header = 0)
orders['together_time'] = str(date)+orders['TIME'].apply(str)
orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
del orders['together_time']

CPU times: user 7min 29s, sys: 10.3 s, total: 7min 39s
Wall time: 8min 39s


In [12]:
# читаем список выявленных айсберг-заявок + преобразуем время в формат datetime
# iceberg_all - список всех найденных айсбергов
# ice_instr - список айсбергов по одному инструменту за один день
iceberg_all = pd.read_csv('iceberg.csv', header = 0)
iceberg_all['together_time'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME'].apply(str)
iceberg_all['together_time_identified'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME_IDENT'].apply(str)
iceberg_all['TIME'] = [ReverseToDateTime(iceberg_all.together_time[i]) for i in range (len(iceberg_all.together_time))]
iceberg_all['TIME_IDENT'] = [ReverseToDateTime(iceberg_all.together_time_identified[i]) for i in range (len(iceberg_all.together_time_identified))]
iceberg_all['DELTA'] = iceberg_all['TIME_IDENT'] - iceberg_all['TIME']
del iceberg_all['together_time']
del iceberg_all['together_time_identified']

# (17.6) Расчет цепочек для одного инструмента внутри одного дня

In [80]:
orders_instr = orders[orders.SECCODE == ticker].reset_index()
orders_instr

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2297,2298,YNDX,S,2019-03-04 10:00:00.000000,2298,1,2315.0,50,NaN,NaN
1,2957,2958,YNDX,S,2019-03-04 10:00:00.000000,2958,1,2300.0,50,NaN,NaN
2,3637,3638,YNDX,S,2019-03-04 10:00:00.000000,3638,1,2298.8,18,NaN,NaN
3,4577,4578,YNDX,S,2019-03-04 10:00:00.000000,4578,1,2449.8,1,NaN,NaN
4,4600,4601,YNDX,S,2019-03-04 10:00:00.000000,4601,1,2399.8,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
129855,5424251,5424252,YNDX,S,2019-03-04 18:44:44.459528,2357504,0,2317.4,5,NaN,NaN
129856,5424260,5424261,YNDX,B,2019-03-04 18:44:48.444374,2565342,0,2231.0,92,NaN,NaN
129857,5424328,5424329,YNDX,B,2019-03-04 18:44:59.238014,2196857,0,2297.6,29,NaN,NaN
129858,5424342,5424343,YNDX,B,2019-03-04 18:44:59.243531,2192515,0,2296.4,25,NaN,NaN


In [127]:
ice_instr = iceberg_all[(iceberg_all.SECCODE == ticker) & (iceberg_all.DATE == date)]
ice_instr 

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,CHAIN_ID
4573,0,B,20190305,1,87698,2285.2,YNDX,2019-03-05 10:01:10.287006,2019-03-05 10:18:40.750123,-400,100,00:17:30.463117,0
4574,1,B,20190305,1,99596,2275.2,YNDX,2019-03-05 10:01:47.844531,2019-03-05 11:50:19.645979,-400,100,01:48:31.801448,0
4575,2,B,20190305,1,124637,2290.2,YNDX,2019-03-05 10:03:25.996026,2019-03-05 10:03:25.996026,-5,100,00:00:00,0
4576,3,B,20190305,1,124639,2290.2,YNDX,2019-03-05 10:03:25.996061,2019-03-05 10:03:25.996096,-90,95,00:00:00.000035,0
4577,4,B,20190305,1,124642,2290.2,YNDX,2019-03-05 10:03:25.996352,2019-03-05 10:03:25.996651,-149,10,00:00:00.000299,0
4578,5,B,20190305,1,873624,2280.2,YNDX,2019-03-05 11:40:46.336752,2019-03-05 11:49:04.716842,-400,100,00:08:18.380090,0
4579,6,B,20190305,1,899500,2281.6,YNDX,2019-03-05 11:44:02.797919,2019-03-05 11:49:04.716842,-400,100,00:05:01.918923,0
4580,7,B,20190305,1,955982,2277.2,YNDX,2019-03-05 11:49:56.378447,2019-03-05 11:49:56.378447,-400,100,00:00:00,0
4581,8,B,20190305,1,966933,2276.0,YNDX,2019-03-05 11:51:52.892656,2019-03-05 11:51:52.892656,-900,100,00:00:00,0
4582,9,B,20190305,1,1079738,2275.0,YNDX,2019-03-05 12:08:54.637858,2019-03-05 12:16:35.052049,-74,100,00:07:40.414191,0


In [83]:
%%time

chains = []
ice_instr['CHAIN_ID'] = 0

for i in range(len(ice_instr)):
    
    id = ice_instr.index[i]
    print(id, ice_instr.loc[id, 'CHAIN_ID'])
    
    # проверяем
    if ice_instr.loc[id, 'CHAIN_ID'] == 0:
    
        buysell = iceberg_all.loc[id][1]
        date = iceberg_all.loc[id][2]
        orderno = iceberg_all.loc[id][4]
        price = iceberg_all.loc[id][5]
        ticker = iceberg_all.loc[id][6]
        time_place = iceberg_all.loc[id][7]
        time_ident = iceberg_all.loc[id][8]
        volume_ini = iceberg_all.loc[id][10]

        left_border = ReverseToDateTime('20190304100000000000')
        right_border = ReverseToDateTime('20190304184500000000')

        # отбираем все заявки с параметрами айсберга (кроме объема), сделки по которым произошли в заданном интервале
        # test - список всех заявок с параматерами айсберга (кроме объема), сделки по которым произошли в заданном интервале
        test = orders_instr[((orders_instr.BUYSELL == buysell) & 
                     (orders_instr.PRICE == price) & 
                     (orders_instr.TIME >= left_border)& 
                     (orders_instr.TIME <= time_ident)&
                     (orders_instr.ORDERNO < orderno)&
                     (orders_instr.ACTION == 2)     
                    )]['ORDERNO'].unique().tolist()

        # проверяем, что исходный объем найденных заявок совпадает с видимой частью айсберга
        # test1 - список всех заявок с параматерами айсберга, сделки по которым произошли в заданном интервале
        test1 = []

        for i in test:

            trade_orderno = i
            order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]

            if order_volume == volume_ini:

                test1.append(i)

        # дописываем номер айсберг заявки в найденную цепочку
        test1.append(orderno)
        # разворачиваем список заявок от самой новой до самой старой
        test1 = test1[::-1]

        # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
        chain_prev = ChainPairPrev(test1)

        right_border = ReverseToDateTime('20190304184500000000')

        # отбираем все заявки с параметрами айсберга (кроме объема), сделки по которым произошли в заданном интервале
        # train - список всех заявок с параматерами айсберга (кроме объема), сделки по которым произошли в заданном интервале
        train = orders_instr[((orders_instr.BUYSELL == buysell) & 
                     (orders_instr.PRICE == price) & 
                     (orders_instr.TIME >= time_ident)& 
                     (orders_instr.TIME <= right_border)&
                     (orders_instr.ORDERNO > orderno)&
                     (orders_instr.ACTION == 1)     
                    )]['ORDERNO'].unique().tolist()

        # проверяем, что исходный объем найденных заявок совпадает с видимой частью айсберга + проверяем, что по всем найденным заявкам была сделка
        # train1 - список всех заявок с параматерами айсберга, сделки по которым произошли в заданном интервале
        train1 = [orderno]

        for i in train:

            trade_orderno = i
            order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]
            trade_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 2)]['VOLUME'].sum()

            if (order_volume == volume_ini) and (order_volume <= trade_volume):

                train1.append(i)

        # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
        chain_suc = ChainPairSuc(train1)
        chain_suc

        # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
        chain_suc = ChainPairSuc(train1)

        chain = chain_prev[::-1] + chain_suc[1:len(chain_suc)]

        #print(id, len(chain))

        chains.append([chain])

        # определяем индекс всех айсбергов, которые вошли в цепочку и меняем их chain_id
        ices = ice_instr.index[ice_instr['ORDERNO'].isin(chain)].to_list()

        for k in ices:

            ice_instr.loc[k, 'CHAIN_ID'] = chain[0]  

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1499 0
1500 0
1501 0
1502 0
1503 0
1504 0
1505 0
1506 0
1507 0
1508 0
1509 0
1510 0
1511 0
1512 0
1513 0
1514 0
1515 0
1516 0
1517 0
1518 0
1519 0
1520 0
1521 0
1522 0
1523 0
1524 0
1525 0
1526 0
1527 0
1528 0
1529 0
1530 0
1531 0
1532 0
1533 0
1534 0
1535 0
1536 0
1537 0
1538 0
1539 0
1540 0
1541 0
1542 0
1543 0
1544 0
1545 0
CPU times: user 37 s, sys: 782 ms, total: 37.8 s
Wall time: 45.6 s


# (17.7) Расчет цепочек для всех инструментов внутри одного дня

In [294]:
%%time
# собираем цепочки за день
chains_all = []

for m in tickers:
    
    ticker = m
    print(ticker)
    orders_instr = orders[orders.SECCODE == ticker].reset_index()
    ice_instr = iceberg_all[(iceberg_all.SECCODE == ticker) & (iceberg_all.DATE == date)]
    
    chains = []
    ice_instr['CHAIN_ID'] = 0

    for i in range(len(ice_instr)):

        id = ice_instr.index[i]

        # проверяем
        if ice_instr.loc[id, 'CHAIN_ID'] == 0:

            buysell = iceberg_all.loc[id][1]
            date = iceberg_all.loc[id][2]
            orderno = iceberg_all.loc[id][4]
            price = iceberg_all.loc[id][5]
            ticker = iceberg_all.loc[id][6]
            time_place = iceberg_all.loc[id][7]
            time_ident = iceberg_all.loc[id][8]
            volume_ini = iceberg_all.loc[id][10]

            left_border = ReverseToDateTime('20190304100000000000')
            right_border = ReverseToDateTime('20190304184500000000')

            # отбираем все заявки с параметрами айсберга (кроме объема), сделки по которым произошли в заданном интервале
            # test - список всех заявок с параматерами айсберга (кроме объема), сделки по которым произошли в заданном интервале
            test = orders_instr[((orders_instr.BUYSELL == buysell) & 
                         (orders_instr.PRICE == price) & 
                         (orders_instr.TIME >= left_border)& 
                         (orders_instr.TIME <= time_ident)&
                         (orders_instr.ORDERNO < orderno)&
                         (orders_instr.ACTION == 2)     
                        )]['ORDERNO'].unique().tolist()

            # проверяем, что исходный объем найденных заявок совпадает с видимой частью айсберга
            # test1 - список всех заявок с параматерами айсберга, сделки по которым произошли в заданном интервале
            test1 = []

            for i in test:

                trade_orderno = i
                order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]

                if order_volume == volume_ini:

                    test1.append(i)

            # дописываем номер айсберг заявки в найденную цепочку
            test1.append(orderno)
            # разворачиваем список заявок от самой новой до самой старой
            test1 = test1[::-1]

            # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
            chain_prev = ChainPairPrev(test1)

            right_border = ReverseToDateTime('20190304184500000000')

            # отбираем все заявки с параметрами айсберга (кроме объема), сделки по которым произошли в заданном интервале
            # train - список всех заявок с параматерами айсберга (кроме объема), сделки по которым произошли в заданном интервале
            train = orders_instr[((orders_instr.BUYSELL == buysell) & 
                         (orders_instr.PRICE == price) & 
                         (orders_instr.TIME >= time_ident)& 
                         (orders_instr.TIME <= right_border)&
                         (orders_instr.ORDERNO > orderno)&
                         (orders_instr.ACTION == 1)     
                        )]['ORDERNO'].unique().tolist()

            # проверяем, что исходный объем найденных заявок совпадает с видимой частью айсберга + проверяем, что по всем найденным заявкам была сделка
            # train1 - список всех заявок с параматерами айсберга, сделки по которым произошли в заданном интервале
            train1 = [orderno]

            for i in train:

                trade_orderno = i
                order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]
                trade_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 2)]['VOLUME'].sum()

                if (order_volume == volume_ini) and (order_volume <= trade_volume):

                    train1.append(i)

            # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
            chain_suc = ChainPairSuc(train1)
            chain_suc

            # проверяем, что время между выставлениями заявок в цепочке меньше, чем заданная ранее дельта и создаем список из заявок, которые удовлетворяют этому условию
            chain_suc = ChainPairSuc(train1)

            chain = chain_prev[::-1] + chain_suc[1:len(chain_suc)]

            #print(id, len(chain))

            chains.append([chain])

            # определяем индекс всех айсбергов, которые вошли в цепочку и меняем их chain_id в ice_instr
            ices = ice_instr.index[(ice_instr['ORDERNO'].isin(chain)) & (ice_instr.DATE == date)].to_list()

            for k in ices:

                ice_instr.loc[k, 'CHAIN_ID'] = chain[0]
            
            # определяем индекс всех айсбергов, которые вошли в цепочку и меняем их chain_id в iceberg_all
            ices1 = iceberg_all.index[(iceberg_all['ORDERNO'].isin(chain)) & (iceberg_all.DATE == date)].to_list()
            
            for l in ices1:
                
                iceberg_all.loc[l, 'CHAIN_ID'] = chain[0]
    
    print(1)
    chains_all.append([ticker, chains])

GAZP


/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


1
SBER
1
GMKN
1
LKOH
1
MTSS
1
MGNT
1
TATN
1
NVTK
1
YNDX
1
ROSN
1
FIVE
1
VTBR
1
SNGS
1
CHMF
1
ALRS
1
CPU times: user 30min 7s, sys: 30.5 s, total: 30min 37s
Wall time: 9h 36min 11s


In [300]:
# сливаем данные по всем цепочки в один список
chains_per_day = [chains_all_04,
                  chains_all_05,
                  chains_all_06,
                  chains_all_07,
                  chains_all_11,
                  chains_all_12,
                  chains_all_13,
                  chains_all_14,
                  chains_all_15,
                  chains_all_18,
                  chains_all_19,
                  chains_all_20,
                  chains_all_21,
                  chains_all_22,
                  chains_all_25,
                  chains_all_26,
                  chains_all_27,
                  chains_all_28,
                  chains_all_29
                 ]

In [318]:
# chains_per_day - список всех цепочек за все дни
# chains_per_day[0] - список цепочек по всем инструментам за первый день
# chains_per_day[0][0][0] - инструмент цепочки (НОЛЬ НЕ МЕНЯТЬ)
# chains_per_day[0][0][1] - цепочка по инструменту (ЕДИНИЦУ НЕ МЕНЯТЬ)
# chains_per_day[0][0][1][4][0] - пятая цепочка по инструменту
chains_per_day[0][0][1][4][0][0] - первый элемент пятой цепочки по инструменту

283520

In [356]:
# собираем данные по найденным цепочкам в датафрейм и сохраняем его для возможности дальнейшей работы
dates1 = []
tickers1 = []
orders1 = []
chains1 = []

for a in range(len(orderlogs)):
    
    date = int(orderlogs[a][8:16])
    
    for b in range(len(tickers)):

        ticker = tickers[b]
        
        for c in range(len(chains_per_day[a][b][1])):
            
            chain = chains_per_day[a][b][1][c][0][0]
            
            for d in range(len(chains_per_day[a][b][1][c][0])):
                
                dates1.append(date)
                tickers1.append(ticker)
                orders1.append(chains_per_day[a][b][1][c][0][d])
                chains1.append(chain)
                
chain_results = pd.DataFrame({'DATE':pd.Series(dates1),
                  'TICKER':pd.Series(tickers1),
                  'ORDERNO':pd.Series(orders1),
                  'CHAIN_ID':pd.Series(chains1)                  
                 })

chain_results.to_csv('chain_results', index = 0)
chain_results

20190304
20190304 GAZP
20190304 GAZP 199077
20190304 GAZP 359638
20190304 GAZP 397852
20190304 GAZP 486370
20190304 GAZP 283520
20190304 GAZP 560068
20190304 GAZP 560069
20190304 GAZP 626087
20190304 GAZP 693576
20190304 GAZP 707548
20190304 GAZP 707551
20190304 GAZP 769025
20190304 GAZP 871524
20190304 GAZP 881498
20190304 GAZP 886166
20190304 GAZP 886174
20190304 GAZP 914062
20190304 GAZP 914065
20190304 GAZP 914066
20190304 GAZP 914067
20190304 GAZP 914068
20190304 GAZP 914069
20190304 GAZP 926050
20190304 GAZP 1035425
20190304 GAZP 1052785
20190304 GAZP 1087211
20190304 GAZP 1104411
20190304 GAZP 1104413
20190304 GAZP 1104416
20190304 GAZP 1106406
20190304 GAZP 1107098
20190304 GAZP 1109306
20190304 GAZP 1108892
20190304 GAZP 1110468
20190304 GAZP 1110474
20190304 GAZP 1110622
20190304 GAZP 1117143
20190304 GAZP 1117214
20190304 GAZP 1117233
20190304 GAZP 1117768
20190304 GAZP 1118096
20190304 GAZP 1118510
20190304 GAZP 1118654
20190304 GAZP 1119094
20190304 GAZP 1119112
20190304 G

20190304 SBER 659640
20190304 SBER 661327
20190304 SBER 668800
20190304 SBER 677828
20190304 SBER 678321
20190304 SBER 679103
20190304 SBER 684525
20190304 SBER 685967
20190304 SBER 691198
20190304 SBER 695265
20190304 SBER 712274
20190304 SBER 712772
20190304 SBER 724509
20190304 SBER 739114
20190304 SBER 739583
20190304 SBER 714049
20190304 SBER 745727
20190304 SBER 751062
20190304 SBER 753732
20190304 SBER 753737
20190304 SBER 753749
20190304 SBER 765706
20190304 SBER 766349
20190304 SBER 768223
20190304 SBER 772326
20190304 SBER 772944
20190304 SBER 783918
20190304 SBER 785391
20190304 SBER 785896
20190304 SBER 791651
20190304 SBER 793690
20190304 SBER 793691
20190304 SBER 793747
20190304 SBER 793822
20190304 SBER 793887
20190304 SBER 795425
20190304 SBER 797463
20190304 SBER 801571
20190304 SBER 805649
20190304 SBER 807629
20190304 SBER 807667
20190304 SBER 807937
20190304 SBER 807946
20190304 SBER 807974
20190304 SBER 807995
20190304 SBER 808004
20190304 SBER 808021
20190304 SBER

20190304 LKOH 1738308
20190304 LKOH 1738309
20190304 LKOH 1738310
20190304 LKOH 1738311
20190304 LKOH 1738323
20190304 LKOH 1763820
20190304 LKOH 1766803
20190304 LKOH 1766837
20190304 LKOH 1766854
20190304 LKOH 1766867
20190304 LKOH 1766871
20190304 LKOH 1766880
20190304 LKOH 1766891
20190304 LKOH 1766944
20190304 LKOH 1778601
20190304 LKOH 1780926
20190304 LKOH 1785166
20190304 LKOH 1785169
20190304 LKOH 1785202
20190304 LKOH 1785218
20190304 LKOH 1785228
20190304 LKOH 1786498
20190304 LKOH 1793155
20190304 LKOH 1800445
20190304 LKOH 1800952
20190304 LKOH 1800959
20190304 LKOH 1800964
20190304 LKOH 1800970
20190304 LKOH 1800975
20190304 LKOH 1800993
20190304 LKOH 1804166
20190304 LKOH 1804167
20190304 LKOH 1804168
20190304 LKOH 1806402
20190304 LKOH 1806383
20190304 LKOH 1882474
20190304 LKOH 2038320
20190304 LKOH 2038326
20190304 LKOH 2050739
20190304 LKOH 2050740
20190304 LKOH 2050741
20190304 LKOH 2075185
20190304 LKOH 2076373
20190304 LKOH 2076374
20190304 LKOH 2076375
20190304 L

20190304 ALRS 1165666
20190304 ALRS 1165667
20190304 ALRS 1165668
20190304 ALRS 1165669
20190304 ALRS 1165671
20190304 ALRS 1165672
20190304 ALRS 1165673
20190304 ALRS 1165676
20190304 ALRS 1165677
20190304 ALRS 1165678
20190304 ALRS 1199765
20190304 ALRS 1214341
20190304 ALRS 1214342
20190304 ALRS 1214343
20190304 ALRS 1214344
20190304 ALRS 1214345
20190304 ALRS 1214346
20190304 ALRS 1214347
20190304 ALRS 1214350
20190304 ALRS 1270547
20190304 ALRS 1277716
20190304 ALRS 1279418
20190304 ALRS 1279419
20190304 ALRS 1279420
20190304 ALRS 1279421
20190304 ALRS 1279422
20190304 ALRS 1279423
20190304 ALRS 1279512
20190304 ALRS 1279836
20190304 ALRS 1312482
20190304 ALRS 1324944
20190304 ALRS 1324947
20190304 ALRS 1326038
20190304 ALRS 1326675
20190304 ALRS 1326873
20190304 ALRS 1344331
20190304 ALRS 1330566
20190304 ALRS 1363954
20190304 ALRS 1372038
20190304 ALRS 1379692
20190304 ALRS 1381039
20190304 ALRS 1416732
20190304 ALRS 1425953
20190304 ALRS 1426053
20190304 ALRS 1426574
20190304 A

20190305 SBER 232586
20190305 SBER 232668
20190305 SBER 232688
20190305 SBER 232694
20190305 SBER 232668
20190305 SBER 236533
20190305 SBER 246842
20190305 SBER 246917
20190305 SBER 248694
20190305 SBER 247805
20190305 SBER 247822
20190305 SBER 250187
20190305 SBER 250187
20190305 SBER 250190
20190305 SBER 250405
20190305 SBER 250452
20190305 SBER 250481
20190305 SBER 252602
20190305 SBER 255048
20190305 SBER 255329
20190305 SBER 255436
20190305 SBER 255639
20190305 SBER 255684
20190305 SBER 255684
20190305 SBER 255763
20190305 SBER 255684
20190305 SBER 255763
20190305 SBER 255763
20190305 SBER 255763
20190305 SBER 255763
20190305 SBER 255763
20190305 SBER 255777
20190305 SBER 255777
20190305 SBER 255777
20190305 SBER 255818
20190305 SBER 255818
20190305 SBER 255835
20190305 SBER 255854
20190305 SBER 255854
20190305 SBER 255854
20190305 SBER 255854
20190305 SBER 255943
20190305 SBER 255977
20190305 SBER 256052
20190305 SBER 256178
20190305 SBER 256557
20190305 SBER 256563
20190305 SBER

...

20190329 SBER 980252
20190329 SBER 1030188
20190329 SBER 1079565
20190329 SBER 1079565
20190329 SBER 1079580
20190329 SBER 1079584
20190329 SBER 1079598
20190329 SBER 1079653
20190329 SBER 1079693
20190329 SBER 1079789
20190329 SBER 1079916
20190329 SBER 1079941
20190329 SBER 1079968
20190329 SBER 1080005
20190329 SBER 1080020
20190329 SBER 1080038
20190329 SBER 1080056
20190329 SBER 1080082
20190329 SBER 1080105
20190329 SBER 1080157
20190329 SBER 1114256
20190329 SBER 1119000
20190329 SBER 1119026
20190329 SBER 1119039
20190329 SBER 1119053
20190329 SBER 1119062
20190329 SBER 1115456
20190329 SBER 1122036
20190329 SBER 1122036
20190329 SBER 1133769
20190329 SBER 1133770
20190329 SBER 1133771
20190329 SBER 1133772
20190329 SBER 1135891
20190329 SBER 1283107
20190329 SBER 1283147
20190329 SBER 1283193
20190329 SBER 1283223
20190329 SBER 1283384
20190329 SBER 1283399
20190329 SBER 1283406
20190329 SBER 156949
20190329 SBER 1291231
20190329 SBER 1291839
20190329 SBER 1293258
20190329 SBE

20190329 SBER 3746398
20190329 SBER 3751384
20190329 SBER 3752888
20190329 SBER 3753972
20190329 SBER 3754057
20190329 SBER 3754175
20190329 SBER 3754189
20190329 SBER 3754190
20190329 SBER 3754225
20190329 SBER 3755057
20190329 SBER 3755098
20190329 SBER 3755445
20190329 SBER 3756405
20190329 SBER 3756983
20190329 SBER 3757771
20190329 SBER 3758152
20190329 SBER 3758154
20190329 SBER 3758209
20190329 SBER 3758369
20190329 SBER 3758411
20190329 SBER 3758418
20190329 SBER 3758425
20190329 SBER 3758441
20190329 SBER 3765235
20190329 SBER 3765235
20190329 SBER 3765306
20190329 SBER 3765669
20190329 SBER 3765676
20190329 SBER 3767870
20190329 SBER 3767942
20190329 SBER 3776706
20190329 SBER 3777237
20190329 SBER 3778426
20190329 SBER 3779087
20190329 SBER 3779255
20190329 SBER 3779255
20190329 SBER 3779698
20190329 SBER 3780167
20190329 SBER 3788395
20190329 SBER 3788398
20190329 SBER 3789525
20190329 SBER 3789762
20190329 SBER 3789917
20190329 SBER 3795091
20190329 SBER 3800126
20190329 S

20190329 MGNT 915415
20190329 MGNT 915415
20190329 MGNT 915415
20190329 MGNT 915415
20190329 MGNT 934816
20190329 MGNT 934817
20190329 MGNT 934818
20190329 MGNT 1020798
20190329 MGNT 1020799
20190329 MGNT 1020802
20190329 MGNT 1020833
20190329 MGNT 1020890
20190329 MGNT 1024887
20190329 MGNT 1031542
20190329 MGNT 1032162
20190329 MGNT 1034927
20190329 MGNT 1035679
20190329 MGNT 1036069
20190329 MGNT 1785876
20190329 MGNT 1790031
20190329 MGNT 1800283
20190329 MGNT 1800284
20190329 MGNT 1824601
20190329 MGNT 1828819
20190329 MGNT 1840242
20190329 MGNT 1924292
20190329 MGNT 1924293
20190329 MGNT 1926198
20190329 MGNT 1930614
20190329 MGNT 1930891
20190329 MGNT 2157162
20190329 MGNT 2160011
20190329 MGNT 2164601
20190329 MGNT 2164602
20190329 MGNT 2164603
20190329 MGNT 2164601
20190329 MGNT 2230550
20190329 MGNT 2231132
20190329 MGNT 2231343
20190329 MGNT 2231455
20190329 MGNT 2264010
20190329 MGNT 2268938
20190329 MGNT 2268938
20190329 MGNT 2559852
20190329 MGNT 2559853
20190329 MGNT 257

20190329 VTBR 3797041
20190329 VTBR 3797045
20190329 VTBR 3797052
20190329 VTBR 3797055
20190329 VTBR 3824833
20190329 VTBR 3826377
20190329 VTBR 3830800
20190329 VTBR 3830811
20190329 VTBR 3837653
20190329 VTBR 3837665
20190329 SNGS
20190329 SNGS 946267
20190329 SNGS 946268
20190329 SNGS 946269
20190329 SNGS 965579
20190329 SNGS 965580
20190329 SNGS 965581
20190329 SNGS 965582
20190329 SNGS 1928892
20190329 SNGS 1928893
20190329 SNGS 2591414
20190329 SNGS 2597638
20190329 SNGS 2597639
20190329 SNGS 2597638
20190329 SNGS 2607874
20190329 SNGS 2634989
20190329 CHMF
20190329 CHMF 611628
20190329 CHMF 611628
20190329 CHMF 611628
20190329 CHMF 719017
20190329 CHMF 924568
20190329 CHMF 924568
20190329 CHMF 1148562
20190329 CHMF 1148575
20190329 CHMF 2287587
20190329 CHMF 2304437
20190329 CHMF 3782707
20190329 CHMF 3783943
20190329 CHMF 3867103
20190329 CHMF 3870438
20190329 CHMF 3878111
20190329 CHMF 3878112
20190329 CHMF 3878113
20190329 ALRS
20190329 ALRS 202839
20190329 ALRS 204317
20190

,DATE,TICKER,ORDERNO,CHAIN_ID
0,20190304,GAZP,199077,199077
1,20190304,GAZP,359638,359638
2,20190304,GAZP,397852,397852
3,20190304,GAZP,486370,486370
4,20190304,GAZP,283520,283520
...,...,...,...,...
76219,20190329,ALRS,3845888,3845888
76220,20190329,ALRS,3845898,3845888
76221,20190329,ALRS,3845899,3845899
76222,20190329,ALRS,3845904,3845904


In [357]:
chain_results.to_csv('chain_results', index = 0)

In [160]:
%%time
# формируем описательные статистики по собранным цепочкам в разрезе инструмент/день
results = []

for j in range(len(tickers)):

    sum_len = 0
    max_len = 0
    sum_vol = 0
    max_vol = 0
    min_dur = dt.timedelta(0)
    max_dur = dt.timedelta(0)

    for i in range(len(chains_all_04[j][1])):
        
        if len(chains_all_04[j][1]) != 0:

            orders_in_chain = chains_all_04[j][1][i][0] 

            # объем цепочки (объем считать как сумму объемов сделок по ордерно)
            cur_vol = orders[(orders['ORDERNO'].isin(orders_in_chain)) & (orders.ACTION == 2)]['VOLUME'].sum()
            sum_vol = sum_vol + cur_vol

            if cur_vol > max_vol:

                max_vol = cur_vol

            # длина цепочки
            cur_len = len(chains_all_04[j][1][i][0])
            sum_len = sum_len + cur_len

            if cur_len > max_len:

                max_len = cur_len

            # длина жизни цепочки (время исполнения последней заявки в целочки - время выставления первой заявки в цепочке)
            orderno_begin = chains_all_04[j][1][i][0][0]
            orderno_end = chains_all_04[j][1][i][0][-1]
            cur_time_end = orders[(orders.ORDERNO == orderno_end)&(orders.ACTION == 2)]['TIME'].max()
            cur_time_begin = orders[(orders.ORDERNO == orderno_begin)&(orders.ACTION == 1)]['TIME'].min()
            cur_dur = cur_time_end - cur_time_begin

            if cur_dur > max_dur:

                max_dur = cur_dur

        mean_len = sum_len/len(chains_all_04[j][1])
        mean_vol = sum_vol/len(chains_all_04[j][1])

    results.append([mean_len, max_len, mean_vol, max_vol, max_dur])

CPU times: user 5min 28s, sys: 33.2 s, total: 6min 1s
Wall time: 3min 22s


In [162]:
%%time
mean_len = []
max_len = []
mean_vol = []
max_vol = []
max_dur = []

for i in range(len(results)):
    
    mean_len.append(results[i][0])
    max_len.append(results[i][1])
    mean_vol.append(results[i][2])
    max_vol.append(results[i][3])
    max_dur.append(results[i][4])

df1 = pd.DataFrame({'SECCODE':pd.Series(tickers),
                  'MEAN_LEN':pd.Series(mean_len),
                  'MAX_LEN':pd.Series(max_len),
                  'MEAN_VOL':pd.Series(mean_vol),
                  'MAX_VOL':pd.Series(max_vol),
                  'MAX_DUR':pd.Series(max_dur)                  
                 })
df1['DATE'] = 20190403

df1

,SECCODE,MEAN_LEN,MAX_LEN,MEAN_VOL,MAX_VOL,MAX_DUR,DATE
0,GAZP,1.555556,11,219.091503,10000,00:55:02.811063,20190403
1,SBER,1.424451,17,5276.854396,1500000,06:58:18.016289,20190403
2,GMKN,1.279279,8,4382.621622,310000,02:21:48.565354,20190403
3,LKOH,1.316406,7,28211.746094,5730000,06:25:53.099388,20190403
4,MTSS,1.407407,4,220.370370,3150,00:13:23.587754,20190403
5,MGNT,1.591837,7,9467.040816,420182,00:18:54.725578,20190403
6,TATN,1.591837,0,9467.040816,0,00:00:00,20190403
7,NVTK,1.111111,3,898.388889,16170,00:00:00,20190403
8,YNDX,1.063830,3,16691.595745,777770,00:00:32.880432,20190403
9,ROSN,1.250000,2,41.500000,600,00:00:00,20190403


In [181]:
%%time
results_all = []

for i in chains_per_day:
    
    print(1, len(chains))
    results = []

    for j in range(len(tickers)):
        
        print(len(chains),j)
        sum_len = 0
        max_len = 0
        sum_vol = 0
        max_vol = 0
        min_dur = dt.timedelta(0)
        max_dur = dt.timedelta(0)

        for i in range(len(chains[j][1])):

            print(len(chains),j,i)
            if len(chains[j][1]) != 0:

                orders_in_chain = chains[j][1][i][0] 

                # объем цепочки (объем считать как сумму объемов сделок по ордерно)
                cur_vol = orders[(orders['ORDERNO'].isin(orders_in_chain)) & (orders.ACTION == 2)]['VOLUME'].sum()
                sum_vol = sum_vol + cur_vol

                if cur_vol > max_vol:

                    max_vol = cur_vol

                # длина цепочки
                cur_len = len(chains[j][1][i][0])
                sum_len = sum_len + cur_len

                if cur_len > max_len:

                    max_len = cur_len

                # длина жизни цепочки (время исполнения последней заявки в целочки - время выставления первой заявки в цепочке)
                orderno_begin = chains[j][1][i][0][0]
                orderno_end = chains[j][1][i][0][-1]
                cur_time_end = orders[(orders.ORDERNO == orderno_end)&(orders.ACTION == 2)]['TIME'].max()
                cur_time_begin = orders[(orders.ORDERNO == orderno_begin)&(orders.ACTION == 1)]['TIME'].min()
                cur_dur = cur_time_end - cur_time_begin

                if cur_dur > max_dur:

                    max_dur = cur_dur

            mean_len = sum_len/len(chains[j][1])
            mean_vol = sum_vol/len(chains[j][1])

        results.append([mean_len, max_len, mean_vol, max_vol, max_dur])
    
    results_all.append(results)

1 15
15 0
15 0 0
15 0 1
15 0 2
15 0 3
15 0 4
15 0 5
15 0 6
15 0 7
15 0 8
15 0 9
15 0 10
15 0 11
15 0 12
15 0 13
15 0 14
15 0 15
15 0 16
15 0 17
15 0 18
15 0 19
15 0 20
15 0 21
15 0 22
15 0 23
15 0 24
15 0 25
15 0 26
15 0 27
15 0 28
15 0 29
15 0 30
15 0 31
15 0 32
15 0 33
15 0 34
15 0 35
15 0 36
15 0 37
15 0 38
15 0 39
15 0 40
15 0 41
15 0 42
15 0 43
15 0 44
15 0 45
15 0 46
15 0 47
15 0 48
15 0 49
15 0 50
15 0 51
15 0 52
15 0 53
15 0 54
15 0 55
15 0 56
15 0 57
15 0 58
15 0 59
15 0 60
15 0 61
15 0 62
15 0 63
15 0 64
15 0 65
15 0 66
15 0 67
15 0 68
15 0 69
15 0 70
15 0 71
15 0 72
15 0 73
15 0 74
15 0 75
15 0 76
15 0 77
15 0 78
15 0 79
15 0 80
15 0 81
15 0 82
15 0 83
15 0 84
15 0 85
15 0 86
15 0 87
15 0 88
15 0 89
15 0 90
15 0 91
15 0 92
15 0 93
15 0 94
15 0 95
15 0 96
15 0 97
15 0 98
15 0 99
15 0 100
15 0 101
15 0 102
15 0 103
15 0 104
15 0 105
15 0 106
15 0 107
15 0 108
15 0 109
15 0 110
15 0 111
15 0 112
15 0 113
15 0 114
15 0 115
15 0 116
15 0 117
15 0 118
15 0 119
15 0 120
15 0 121
15

15 2 60
15 2 61
15 2 62
15 2 63
15 2 64
15 2 65
15 2 66
15 2 67
15 2 68
15 2 69
15 2 70
15 2 71
15 2 72
15 2 73
15 2 74
15 2 75
15 2 76
15 2 77
15 2 78
15 2 79
15 2 80
15 2 81
15 2 82
15 2 83
15 2 84
15 2 85
15 2 86
15 2 87
15 2 88
15 2 89
15 2 90
15 2 91
15 2 92
15 2 93
15 2 94
15 2 95
15 2 96
15 2 97
15 2 98
15 2 99
15 2 100
15 2 101
15 2 102
15 2 103
15 2 104
15 2 105
15 2 106
15 2 107
15 2 108
15 2 109
15 2 110
15 3
15 3 0
15 3 1
15 3 2
15 3 3
15 3 4
15 3 5
15 3 6
15 3 7
15 3 8
15 3 9
15 3 10
15 3 11
15 3 12
15 3 13
15 3 14
15 3 15
15 3 16
15 3 17
15 3 18
15 3 19
15 3 20
15 3 21
15 3 22
15 3 23
15 3 24
15 3 25
15 3 26
15 3 27
15 3 28
15 3 29
15 3 30
15 3 31
15 3 32
15 3 33
15 3 34
15 3 35
15 3 36
15 3 37
15 3 38
15 3 39
15 3 40
15 3 41
15 3 42
15 3 43
15 3 44
15 3 45
15 3 46
15 3 47
15 3 48
15 3 49
15 3 50
15 3 51
15 3 52
15 3 53
15 3 54
15 3 55
15 3 56
15 3 57
15 3 58
15 3 59
15 3 60
15 3 61
15 3 62
15 3 63
15 3 64
15 3 65
15 3 66
15 3 67
15 3 68
15 3 69
15 3 70
15 3 71
15 3 72
15

15 14 295
15 14 296
15 14 297
15 14 298
15 14 299
15 14 300
15 14 301
15 14 302
15 14 303
15 14 304
15 14 305
15 14 306
15 14 307
15 14 308
15 14 309
15 14 310
15 14 311
15 14 312
15 14 313
15 14 314
15 14 315
15 14 316
15 14 317
15 14 318
15 14 319
15 14 320
15 14 321
15 14 322
15 14 323
15 14 324
15 14 325
15 14 326
15 14 327
15 14 328
15 14 329
15 14 330
15 14 331
15 14 332
15 14 333
15 14 334
15 14 335
15 14 336
15 14 337
15 14 338
15 14 339
15 14 340
15 14 341
15 14 342
15 14 343
15 14 344
15 14 345
15 14 346
15 14 347
15 14 348
15 14 349
15 14 350
15 14 351
15 14 352
15 14 353
15 14 354
15 14 355
15 14 356
15 14 357
15 14 358
15 14 359
15 14 360
15 14 361
15 14 362
15 14 363
15 14 364
15 14 365
15 14 366
15 14 367
15 14 368
15 14 369
15 14 370
15 14 371
15 14 372
15 14 373
15 14 374
15 14 375
15 14 376
15 14 377
15 14 378
15 14 379
15 14 380
15 14 381
15 14 382
15 14 383
15 14 384
15 14 385
15 14 386
15 14 387
15 14 388
15 14 389
15 14 390
15 14 391
15 14 392
15 14 393
15 14 394


15 1 362
15 1 363
15 1 364
15 1 365
15 1 366
15 1 367
15 1 368
15 1 369
15 1 370
15 1 371
15 1 372
15 1 373
15 1 374
15 1 375
15 1 376
15 1 377
15 1 378
15 1 379
15 1 380
15 1 381
15 1 382
15 1 383
15 1 384
15 1 385
15 1 386
15 1 387
15 1 388
15 1 389
15 1 390
15 1 391
15 1 392
15 1 393
15 1 394
15 1 395
15 1 396
15 1 397
15 1 398
15 1 399
15 1 400
15 1 401
15 1 402
15 1 403
15 1 404
15 1 405
15 1 406
15 1 407
15 1 408
15 1 409
15 1 410
15 1 411
15 1 412
15 1 413
15 1 414
15 1 415
15 1 416
15 1 417
15 1 418
15 1 419
15 1 420
15 1 421
15 1 422
15 1 423
15 1 424
15 1 425
15 1 426
15 1 427
15 1 428
15 1 429
15 1 430
15 1 431
15 1 432
15 1 433
15 1 434
15 1 435
15 1 436
15 1 437
15 1 438
15 1 439
15 1 440
15 1 441
15 1 442
15 1 443
15 1 444
15 1 445
15 1 446
15 1 447
15 1 448
15 1 449
15 1 450
15 1 451
15 1 452
15 1 453
15 1 454
15 1 455
15 1 456
15 1 457
15 1 458
15 1 459
15 1 460
15 1 461
15 1 462
15 1 463
15 1 464
15 1 465
15 1 466
15 1 467
15 1 468
15 1 469
15 1 470
15 1 471
15 1 472
1

15 1 1246
15 1 1247
15 1 1248
15 1 1249
15 1 1250
15 1 1251
15 1 1252
15 1 1253
15 1 1254
15 1 1255
15 1 1256
15 1 1257
15 1 1258
15 1 1259
15 1 1260
15 1 1261
15 1 1262
15 1 1263
15 1 1264
15 1 1265
15 1 1266
15 1 1267
15 1 1268
15 1 1269
15 1 1270
15 1 1271
15 1 1272
15 1 1273
15 1 1274
15 1 1275
15 1 1276
15 1 1277
15 1 1278
15 1 1279
15 1 1280
15 1 1281
15 1 1282
15 1 1283
15 1 1284
15 1 1285
15 1 1286
15 1 1287
15 1 1288
15 1 1289
15 1 1290
15 1 1291
15 1 1292
15 1 1293
15 1 1294
15 1 1295
15 1 1296
15 1 1297
15 1 1298
15 1 1299
15 1 1300
15 1 1301
15 1 1302
15 1 1303
15 1 1304
15 1 1305
15 1 1306
15 1 1307
15 1 1308
15 1 1309
15 1 1310
15 1 1311
15 1 1312
15 1 1313
15 1 1314
15 1 1315
15 1 1316
15 1 1317
15 1 1318
15 1 1319
15 1 1320
15 1 1321
15 1 1322
15 1 1323
15 1 1324
15 1 1325
15 1 1326
15 1 1327
15 1 1328
15 1 1329
15 1 1330
15 1 1331
15 1 1332
15 1 1333
15 1 1334
15 1 1335
15 1 1336
15 1 1337
15 1 1338
15 1 1339
15 1 1340
15 1 1341
15 1 1342
15 1 1343
15 1 1344
15 1 1345


15 14 80
15 14 81
15 14 82
15 14 83
15 14 84
15 14 85
15 14 86
15 14 87
15 14 88
15 14 89
15 14 90
15 14 91
15 14 92
15 14 93
15 14 94
15 14 95
15 14 96
15 14 97
15 14 98
15 14 99
15 14 100
15 14 101
15 14 102
15 14 103
15 14 104
15 14 105
15 14 106
15 14 107
15 14 108
15 14 109
15 14 110
15 14 111
15 14 112
15 14 113
15 14 114
15 14 115
15 14 116
15 14 117
15 14 118
15 14 119
15 14 120
15 14 121
15 14 122
15 14 123
15 14 124
15 14 125
15 14 126
15 14 127
15 14 128
15 14 129
15 14 130
15 14 131
15 14 132
15 14 133
15 14 134
15 14 135
15 14 136
15 14 137
15 14 138
15 14 139
15 14 140
15 14 141
15 14 142
15 14 143
15 14 144
15 14 145
15 14 146
15 14 147
15 14 148
15 14 149
15 14 150
15 14 151
15 14 152
15 14 153
15 14 154
15 14 155
15 14 156
15 14 157
15 14 158
15 14 159
15 14 160
15 14 161
15 14 162
15 14 163
15 14 164
15 14 165
15 14 166
15 14 167
15 14 168
15 14 169
15 14 170
15 14 171
15 14 172
15 14 173
15 14 174
15 14 175
15 14 176
15 14 177
15 14 178
15 14 179
15 14 180
15 14 181


15 1 362
15 1 363
15 1 364
15 1 365
15 1 366
15 1 367
15 1 368
15 1 369
15 1 370
15 1 371
15 1 372
15 1 373
15 1 374
15 1 375
15 1 376
15 1 377
15 1 378
15 1 379
15 1 380
15 1 381
15 1 382
15 1 383
15 1 384
15 1 385
15 1 386
15 1 387
15 1 388
15 1 389
15 1 390
15 1 391
15 1 392
15 1 393
15 1 394
15 1 395
15 1 396
15 1 397
15 1 398
15 1 399
15 1 400
15 1 401
15 1 402
15 1 403
15 1 404
15 1 405
15 1 406
15 1 407
15 1 408
15 1 409
15 1 410
15 1 411
15 1 412
15 1 413
15 1 414
15 1 415
15 1 416
15 1 417
15 1 418
15 1 419
15 1 420
15 1 421
15 1 422
15 1 423
15 1 424
15 1 425
15 1 426
15 1 427
15 1 428
15 1 429
15 1 430
15 1 431
15 1 432
15 1 433
15 1 434
15 1 435
15 1 436
15 1 437
15 1 438
15 1 439
15 1 440
15 1 441
15 1 442
15 1 443
15 1 444
15 1 445
15 1 446
15 1 447
15 1 448
15 1 449
15 1 450
15 1 451
15 1 452
15 1 453
15 1 454
15 1 455
15 1 456
15 1 457
15 1 458
15 1 459
15 1 460
15 1 461
15 1 462
15 1 463
15 1 464
15 1 465
15 1 466
15 1 467
15 1 468
15 1 469
15 1 470
15 1 471
15 1 472
1

KeyboardInterrupt: 